<a href="https://colab.research.google.com/github/10Yash29/Plagiarism/blob/main/maverick_hackx.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers torch

In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) y
Token is valid (permission: write).
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in 

In [ ]:
from transformers import BertTokenizer, BertModel
from transformers.utils import logging

logging.set_verbosity_warning()
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', use_auth_token=True)
model = BertModel.from_pretrained('bert-base-uncased', use_auth_token=True)


In [ ]:
import os
import torch
from transformers import BertTokenizer, BertModel
from sklearn.metrics.pairwise import cosine_similarity

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')
model.eval()

def get_embedding(text):
    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True, max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)
    last_hidden_states = outputs.last_hidden_state
    embeddings = last_hidden_states.mean(dim=1)
    return embeddings

DATABASE_DIR = '/content/training data/original_1'

def load_training_data():
    if not os.path.exists(DATABASE_DIR):
        raise FileNotFoundError(f"The directory {DATABASE_DIR} does not exist.")

    data = []
    labels = []


    for filename in os.listdir(DATABASE_DIR):
        file_path = os.path.join(DATABASE_DIR, filename)
        with open(file_path, 'r') as f:
            code = f.read().strip()

            if len(code) == 0:
                print(f"Skipping empty file: {filename}")
                continue

            data.append(code)

            if 'original' in filename:
                labels.append(0)
            else:
                labels.append(1)

    if len(data) == 0:
        raise ValueError("No valid code files found.")

    return data, labels

try:
    data, labels = load_training_data()
except (FileNotFoundError, ValueError) as e:
    print(e)

if data:
    train_embeddings = [get_embedding(code) for code in data]

    # Hardcoded test submission
    test_code = """
    class Solution:
        def searchMatrix(self, matrix: List[List[int]], target: int) -> bool:
            top_row = 0
            bottom_row = len(matrix) - 1
            while top_row <= bottom_row:
                middle_row = (top_row + bottom_row) // 2
                if matrix[middle_row][0] < target and matrix[middle_row][-1] > target:
                    break
                elif matrix[middle_row][0] > target:
                    bottom_row = middle_row - 1
                else:
                    top_row = middle_row + 1
            row_to_search = (top_row + bottom_row) // 2
            left_col = 0
            right_col = len(matrix[row_to_search]) - 1
            while left_col <= right_col:
                middle_col = (left_col + right_col) // 2
                if matrix[row_to_search][middle_col] == target:
                    return True
                elif matrix[row_to_search][middle_col] > target:
                    right_col = middle_col - 1
                else:
                    left_col = middle_col + 1
            return False
    """


    test_embedding = get_embedding(test_code)


    similarities = [cosine_similarity(test_embedding.numpy(), train_embedding.numpy())[0][0] for train_embedding in train_embeddings]

    if max(similarities) > 0.8:
        print("The test submission is plagiarized.")
    else:
        print("The test submission is original.")


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


The test submission is plagiarized.
